# Analysis of Patski data

$\newcommand{\RR}{\mathbb{R}}$
This notebook contains coded for the analysis of the Patski X chromosome dataset (Section 5.2 in the paper).

In [52]:
using HomotopyContinuation, LinearAlgebra, Random, MATLAB, NBInclude

In [53]:
Random.seed!(123)
mat"rng(123,'twister')"

## Load data

Load Matlab file with contact count matrices.

In [54]:
mat"""

load('patski_contact_count_matrices.mat')
n = size(A,1);

"""

## Filtering

Exclude loci with low total contact counts.

In [55]:
mat"""

total_ambig_contacts = nansum(A,2);
total_pa_contacts = nansum(P,1)'+nansum(P(1:n,:),2)+nansum(P(n+(1:n),:),2);
total_ua_contacts = nansum(U(1:n,:),2)+nansum(U((1:n)+n,:),2);

total_contacts = total_ambig_contacts+total_pa_contacts+total_ua_contacts;

[sorted_total_contacts,indices_sorted] = sort(total_contacts);

plot(sorted_total_contacts)
title("The i-th smallest total contact count")
xlabel("i")

set(gcf,'renderer','Painters')
print(gcf,'-depsc','total_contacts')

"""

In [56]:
mat"""

ratios = zeros(1,n-1);

for i = 1:n-1
    if sorted_total_contacts(i)==0
        ratios(i)=NaN;
    else
        ratios(i)=sorted_total_contacts(i+1)/sorted_total_contacts(i);
    end
end

plot(ratios)
title('Ratio between the i-th and (i+1)-th smallest total contact count')
xlabel('i')

set(gcf,'renderer','Painters')
print(gcf,'-depsc','total_contact_ratio')

"""

In [60]:
mat"""

number_to_remove = 47;

removed_pairs = sort(indices_sorted(1:number_to_remove));
filtered_pairs = setdiff(1:n,removed_pairs);

n_filtered = length(filtered_pairs);

plot(total_contacts)
hold on
plot(removed_pairs,total_contacts(removed_pairs),'*k')
hold off
title('Total contacts of the i-th locus')
xlabel('i')

"""

In [61]:
mat"""

U_filtered = U([filtered_pairs,filtered_pairs+n],[filtered_pairs,filtered_pairs+n]);
P_filtered = P([filtered_pairs,filtered_pairs+n],filtered_pairs);
A_filtered = A(filtered_pairs,filtered_pairs);

"""

## Choice of ambiguous loci

Make a choice of which loci we want to view as ambiguous for the rest of the analysis.

In [62]:
mat"""

ambiguity_quotient = ( nansum(A,2)+nansum(P,1)' ) ./ ( total_contacts );
unambiguity_quotient = 1-ambiguity_quotient;
plot(filtered_pairs,ambiguity_quotient(filtered_pairs))
title('Proportion of contacts where i-th locus is unambiguous')

"""

In [64]:
mat"""
unambiguity_quotient_sorted = sort(unambiguity_quotient(filtered_pairs));
plot(unambiguity_quotient_sorted)
title('The i-th smallest unambiguity quotient')
xlabel('i')
%grid()
%xticks(0:10:n_filtered)
set(gcf,'renderer','Painters')
print(gcf,'-depsc','unambiguity_quotient_sorted')
"""

In [14]:
mat"""

ambiguity_threshold = 0.60;

ua_pairs_filtered = find(ambiguity_quotient(filtered_pairs)<ambiguity_threshold);
ambig_pairs_filtered = setdiff(1:length(filtered_pairs),ua_pairs_filtered);

ua_pairs = filtered_pairs(ua_pairs_filtered);
ambig_pairs = filtered_pairs(ambig_pairs_filtered);

m_filtered = length(ambig_pairs);
m = m_filtered;

disp(m)

"""

    46



## Make loci completely ambiguous or unambiguous

Adapt the data to equation (2.2) in the paper, folliwing the strategy described in Supporting Material.

In [17]:
mat"""

[U_filtered_new,P_filtered_new,A_filtered_new] = preprocess_contacts(U_filtered,P_filtered,A_filtered,ambig_pairs_filtered)

"""

## Estimation of unambiguous loci

In [19]:
mat"""

[Xua_filtered,Yua_filtered] = estimate_disambiguated(U_filtered_new,ua_pairs_filtered,...
    optimization_method='chromsde',alpha=-2);

X_filtered = NaN(3,n_filtered);
Y_filtered = NaN(3,n_filtered);

X_filtered(:,ua_pairs_filtered)=Xua_filtered;
Y_filtered(:,ua_pairs_filtered)=Yua_filtered;

plot3(Xua_filtered(1,:),Xua_filtered(2,:),Xua_filtered(3,:),'DisplayName','Inactive homolog')
hold on
plot3(Yua_filtered(1,:),Yua_filtered(2,:),Yua_filtered(3,:),'DisplayName','Active homolog')
hold off

"""


*************************************************************************************
  PPASolver For Symmetric Matrix Problems
  rho_target = -7.38e-01
  continuation = 1
  Newton_alteroption = 0
  scale_data = 2, precond = 3
  normC = 3.8e+02, normA = 4.3e+02, normb = 3.6e+01
  number of initial iterations = 50
  sig0 = 1.00e+01,  iter = 50
  norm_XZratio = 9.9e+00
  orig-normX = 5.5e+02, orig-normZ = 5.6e+00
  normX = 5.6e+01, normZ = 5.6e+01
 ****************************************************************************************************************
  it   prim_inf    dual_inf         prim_obj          dual_obj           relgap      time     sigma      rho      rankX    |AltN_iter   AltN_time
*****************************************************************************************************************
   0  2.7e-02    1.2e-02     -8.9690750e+02    -6.1125586e+02  -3.79e-01   7.4    1.0e+00  7.38e+01    29(I)(I)
       good termination:   gradLy=2.24e-02, priminf_sub=6.19e-0

In [20]:
mat"""

title_str = strcat("ChromSDE estimation");
title(title_str)

legend('Location','southoutside','NumColumns',2)

"""

## Estimation of ambiguous loci using NAG

In [21]:
mat"""

$Zua_filtered = [Xua_filtered,Yua_filtered];
$Pred_filtered = P_filtered_new([ua_pairs_filtered,ua_pairs_filtered+n_filtered],...
    ambig_pairs_filtered);

"""

In [22]:
@nbinclude("NAG_code.ipynb")
Xhtpy_ambig_filtered, Yhtpy_ambig_filtered, T, attempts, ua_choices, minimal_distances = estimate_ambig_htpy(Pred_filtered,Zua_filtered,5,number_of_contacts=20,max_attempts=100,imag_part_threshold=0.15);

Tracking 40 paths... 100%|██████████████████████████████| Time: 0:00:00
  # paths tracked:                  40
  # non-singular solutions (real):  40 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         40 (0)
Locus: 30 of 46
Locus: 31 of 46
Locus: 32 of 46
Locus: 33 of 46
Locus: 34 of 46
Locus: 35 of 46
Locus: 36 of 46
Locus: 37 of 46
Locus: 38 of 46
Locus: 39 of 46
Locus: 40 of 46
Locus: 41 of 46
Locus: 42 of 46
Locus: 43 of 46
Locus: 44 of 46
Locus: 45 of 46
Locus: 46 of 46


In [23]:
display(attempts)
display(maximum(attempts))
display(T)
display(minimal_distances)

5×46 Matrix{Int64}:
 8  6  2  5  4  2  2  1  6  1  4  1  …  2  3   1  5  3  1  4  2  3  1  3  3
 7  1  1  1  2  2  4  1  2  1  3  1     2  2   8  3  3  3  3  1  3  3  2  1
 6  1  2  3  1  3  4  5  3  3  1  3     2  2  12  1  1  2  1  1  2  1  1  1
 7  1  2  5  1  1  2  5  8  6  1  1     5  1   2  4  1  1  1  1  5  4  2  1
 1  2  1  1  1  3  5  3  1  6  6  2     1  2   1  1  1  2  2  4  3  2  1  1

12

90.02600002288818

46-element Vector{Float64}:
 0.2214350481860813
 0.24750091819660983
 0.10000325682346871
 0.05986660649575022
 0.09554876904569978
 0.1550348061812129
 0.13041743003879103
 0.15262613984070547
 0.18676070926587574
 0.11479141737227187
 0.07226891955071763
 0.10938940312978138
 0.04593320359211021
 ⋮
 0.09466448770388344
 0.1615805577442241
 0.16425523918130489
 0.10699589495979796
 0.14215611003038667
 0.06072454879705515
 0.08295878919799105
 0.07433412528851416
 0.13423988879672938
 0.16889809207880574
 0.1562522538766635
 0.10239634903868614

In [24]:
mat"""

Xhtpy_ambig_filtered = $Xhtpy_ambig_filtered;
Yhtpy_ambig_filtered = $Yhtpy_ambig_filtered;

Xhtpy_filtered = zeros(3,n_filtered);
Xhtpy_filtered(:,ua_pairs_filtered) = Xua_filtered;
Xhtpy_filtered(:,ambig_pairs_filtered) = Xhtpy_ambig_filtered;

Yhtpy_filtered = zeros(3,n_filtered);
Yhtpy_filtered(:,ua_pairs_filtered) = Yua_filtered;
Yhtpy_filtered(:,ambig_pairs_filtered) = Yhtpy_ambig_filtered;

"""

In [25]:
mat"""

plot3(Xhtpy_filtered(1,:),Xhtpy_filtered(2,:),Xhtpy_filtered(3,:),'b','DisplayName','X')
hold on
plot3(Yhtpy_filtered(1,:),Yhtpy_filtered(2,:),Yhtpy_filtered(3,:),'r','DisplayName','Y')
plot(NaN,NaN,'*','color','k','DisplayName','Ambiguous')
legend('AutoUpdate','off','Location','southoutside','NumColumns',3)
plot3(Xhtpy_filtered(1,ambig_pairs_filtered),Xhtpy_filtered(2,ambig_pairs_filtered),Xhtpy_filtered(3,ambig_pairs_filtered),'*','color','b')
plot3(Yhtpy_filtered(1,ambig_pairs_filtered),Yhtpy_filtered(2,ambig_pairs_filtered),Yhtpy_filtered(3,ambig_pairs_filtered),'*','color','r')
hold off

"""

## Additional local optimization step

In [26]:
mat"""

[Xesthtpy_filtered,Yesthtpy_filtered,T,fvalbest,exitflagbest,outputbest,gradbest] = estimate_ambig(...
    Xua_filtered,Yua_filtered,ua_pairs_filtered,P_filtered_new,...
    num_initializations=1,initialization_factor=0.0,Xstart=Xhtpy_filtered,Ystart=Yhtpy_filtered,clustering_step=true);

"""


Local minimum found.

Optimization completed because the size of the gradient is less than
the value of the optimality tolerance.

The following pairs were switched: 1   49   50   77   92   93  115  125  129  134  135  141  142  148  149  150  153  156  169  176  187  188  190  191  195  208  221  256 .


In [37]:
mat"""

disp(T)
disp(outputbest)

"""

  295.3663



In [41]:
mat"""

Xesthtpy = NaN(3,n);
Yesthtpy = NaN(3,n);

Xesthtpy(:,filtered_pairs) = Xesthtpy_filtered;
Yesthtpy(:,filtered_pairs) = Yesthtpy_filtered;

"""

In [43]:
mat"""

close all
plot3(Xesthtpy(1,:),Xesthtpy(2,:),Xesthtpy(3,:),'Color','#0072BD','DisplayName','Inactive homolog')
hold on
plot3(Yesthtpy(1,:),Yesthtpy(2,:),Yesthtpy(3,:),'Color','#D95319','DisplayName','Active homolog')

plot(NaN,NaN,'*','color','k','DisplayName','Ambiguous')
legend('AutoUpdate','off','Location','southoutside','NumColumns',3)

plot3(Xesthtpy(1,ambig_pairs),Xesthtpy(2,ambig_pairs),Xesthtpy(3,ambig_pairs),'*','color','#0072BD')
plot3(Yesthtpy(1,ambig_pairs),Yesthtpy(2,ambig_pairs),Yesthtpy(3,ambig_pairs),'*','color','#D95319')

plot3(Xesthtpy_filtered(1,:),Xesthtpy_filtered(2,:),Xesthtpy_filtered(3,:),'Color','#0072BD','Linestyle','--')
plot3(Yesthtpy_filtered(1,:),Yesthtpy_filtered(2,:),Yesthtpy_filtered(3,:),'Color','#D95319','Linestyle','--')

hold off
title_str = strcat("NAG+local, Objective function = ",num2str(fvalbest,'%e'));
view(0,90)

"""

In [44]:
mat"""

set(gcf,'renderer','Painters')
print(gcf,'-depsc','real-data-nag-local')

"""

# Bipartite index

In [52]:
mat"""

close all
BI_diagram(Xesthtpy_filtered,Yesthtpy_filtered,alpha=-2,indices=filtered_pairs,Xname='Inactive homolog',Yname='Active homolog');
legend('AutoUpdate','off','Location','southeast');
hold on;
xline(146,'--');
hold off;
title("")

"""

1×296 Matrix{Float64}:
 2.51388  1.3376  3.16321  8.21006  …  4.04061  3.40994  1.82957  Inf

In [65]:
mat"""

set(gcf,'renderer','Painters')
print(gcf,'-depsc','bi_index')

"""

# Contact count matrices

In [57]:
mat"""

Ared = A_filtered_new(ambig_pairs_filtered,ambig_pairs_filtered);

for i=1:length(ambig_pairs)
    Ared(i,i)=Inf;
end;

Ared_estimated = zeros(length(ambig_pairs_filtered),length(ambig_pairs_filtered));

for i=1:length(ambig_pairs_filtered)
    for j=1:length(ambig_pairs_filtered)
        Ared_estimated(i,j) = 1/norm(Xambig_estimated(:,i)-Xambig_estimated(:,j))^2 + 1/norm(Xambig_estimated(:,i)-Yambig_estimated(:,j))^2 + ...
            1/norm(Yambig_estimated(:,i)-Xambig_estimated(:,j))^2 + 1/norm(Yambig_estimated(:,i)-Yambig_estimated(:,j))^2;
    end
end

imagesc(log10(Ared));
title("Patski data")
colormap(flipud(gray))
caxis([0,3])
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','real_A')

imagesc(log10(Ared_estimated));
title("SNLC reconstruction")
colormap(flipud(gray))
caxis([0,3])
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','reconstructed_A')
"""



In [55]:
mat"""

Pred = P_filtered_new([ua_pairs_filtered,n_filtered+ua_pairs_filtered],ambig_pairs_filtered);
imagesc(log10(Pred));
title("Patski data")
colormap(flipud(gray))
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','real_P')

Pred_estimated = zeros(2*length(ua_pairs_filtered),length(ambig_pairs_filtered));

Zua_estimated = [Xesthtpy_filtered(:,ua_pairs_filtered),Yesthtpy_filtered(:,ua_pairs_filtered)];
Xambig_estimated = Xesthtpy_filtered(:,ambig_pairs_filtered);
Yambig_estimated = Yesthtpy_filtered(:,ambig_pairs_filtered);

for i=1:2*length(ua_pairs_filtered)
    for j=1:length(ambig_pairs_filtered)
        Pred_estimated(i,j) = 1/norm(Zua_estimated(:,i)-Xambig_estimated(:,j))^2 + 1/norm(Zua_estimated(:,i)-Yambig_estimated(:,j))^2;
    end
end

imagesc(log10(Pred));
title("Patski data")
colormap(flipud(gray))
caxis([0,3.5])
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','real_P')

imagesc(log10(Pred_estimated));
title("SNLC reconstruction")
colormap(flipud(gray))
caxis([0,3.5])
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','reconstructed_P')

imagesc(log10(Pred_estimated));
title("SNLC reconstruction")
colormap(flipud(gray))
caxis([0,3.5])
colorbar
"""


In [58]:
mat"""

Ured = U_filtered_new([ua_pairs_filtered,ua_pairs_filtered+n_filtered],[ua_pairs_filtered,ua_pairs_filtered+n_filtered]);

for i=1:2*length(ua_pairs_filtered)
    Ured(i,i)=Inf;
end

Ured_estimated = zeros(2*length(ua_pairs_filtered),2*length(ua_pairs_filtered));

for i=1:2*length(ua_pairs_filtered)
    for j=1:2*length(ua_pairs_filtered)
        Ured_estimated(i,j) = 1/norm(Zua_estimated(:,i)-Zua_estimated(:,j))^2;
    end
end

imagesc(log10(Ured));
title("Patski data")
colormap(flipud(gray))
caxis([0,3.5])
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','real_U')


imagesc(log10(Ured_estimated));
title("SNLC reconstruction")
colormap(flipud(gray))
caxis([0,3.5])
colorbar
set(gcf,'renderer','Painters')
print(gcf,'-depsc','reconstructed_U')

"""

